In [1]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import numpy as np

# Load Configs and Define Circuit

define FPGA config; this has timing information for the scheduler. For now it is fine to use the default config

In [2]:
fpga_config = FPGAConfig()

load channel configs (firmware channel mapping + configuration, see [Understanding Channel Configuration](https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration) for details), and QChip object, which contains calibrated gates + readout.


In [3]:
channel_configs = load_channel_configs('channel_config.json')
qchip = qc.QChip('qubitcfg.json')

As an alternative to the above, if you're using the chipcalibration repository, you can load all three configs like this:

In [ ]:
import chipcalibration.config as cfg
chipname = 'X4Y2/sian' #this is a folder in the 'qchip' submodule of chipcalibration, containing the name of your chip
fpga_config, qchip, channel_config = cfg.load_configs(chipname)

# Example: Simple Amplitude Sweep

This circuit applies a bit flip on `Q0` conditioned on the classical measurement result from `Q1`. Note that no delays need to be inserted between the `read` on `Q1` and the pulses applied to `Q0`; this is resolved by the compiler according to the `FPROCChannel` objects in the `FPGAConfig`.

In [4]:
amp_start = 0.1
amp_stop = 1
amp_step = 0.1

circuit = [
    {'name': 'declare', 'var': 'amp', 'scope': ['Q0'], 'dtype': 'amp'},
    {'name': 'set_var', 'value': amp_start, 'var': 'amp'}, # pulse amplitude is parameterized by processor register
    
    # loop + increment amplitude by amp_step until it reaches amp_stop
    {'name': 'loop', 'cond_lhs': amp_stop, 'alu_cond': 'ge', 'cond_rhs': 'amp', 'scope': ['Q0'], 
     'body': [
         
         {'name': 'delay', 't': 500.e-6},
         
         {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 'amp', 'twidth': 2.4e-08,
             'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
             'dest': 'Q0.qdrv'},
         
         {'name': 'read', 'qubit': 'Q0'},
         
         {'name': 'alu', 'op': 'add', 'lhs': amp_step, 'rhs': 'amp', 'out': 'amp'}


         
     ]}]

## Compile and Assemble

Compile the program. 

In [5]:
compiled_prog = tc.run_compile_stage(circuit, fpga_config, qchip)

Run the assembler to convert the above program into machine code that we can load onto the FPGA:

In [6]:
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

## Connect to Server and Run Circuit

Now that we've defined our circuit and compiled it to machine code, we can submit it to the ZCU216 and run it.

Instantiate the runner client:

# Example: Compound Freq x Amplitude Sweep

In [7]:
amp_start = 0.1
amp_stop = 1
amp_step = 0.1

freqs = np.linspace(4.493e9, 4.495e9, 30) # frequencies can be an arbitrary array, 
                                          # since we're storing them in a table and accessing the address

# first, declare all frequencies
circuit = [{'name': 'declare_freq', 'freq': freq, 'scope': ['Q0.qdrv'], 'freq_ind': i} 
           for i, freq in enumerate(freqs)]

# fill in the rest of the circuit
circuit.extend([
    {'name': 'declare', 'var': 'amp', 'scope': ['Q0'], 'dtype': 'amp'},
    {'name': 'set_var', 'value': amp_start, 'var': 'amp'}, # pulse amplitude is parameterized by processor register
    
    {'name': 'declare', 'var': 'freq_ind', 'scope': ['Q0'], 'dtype': 'int'},
    {'name': 'set_var', 'value': 0, 'var': 'freq_ind'}, # frequency index is parameterized by a register
    
    # outer loop over amplitude
    {'name': 'loop', 'cond_lhs': amp_stop, 'alu_cond': 'ge', 'cond_rhs': 'amp', 'scope': ['Q0'], 
     'body': [
         # inner loop over frequency
         {'name': 'loop', 'cond_lhs': len(freqs), 'alu_cond': 'ge', 'cond_rhs': 'freq_ind', 'scope': ['Q0'], 
              'body': [

                    {'name': 'delay', 't': 500.e-6},
                  
                    {'name': 'pulse', 'phase': 0, 'freq': 'freq_ind', 'amp': 'amp', 'twidth': 2.4e-08,
                         'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
                         'dest': 'Q0.qdrv'},
                  
                    {'name': 'read', 'qubit': 'Q0'},
                
                    {'name': 'alu', 'op': 'add', 'lhs': 1, 'rhs': 'freq_ind', 'out': 'amp'},

                  
            ]},
         {'name': 'alu', 'op': 'add', 'lhs': amp_step, 'rhs': 'amp', 'out': 'amp'},

        ]
    }
])

In [8]:
compiled_prog = tc.run_compile_stage(circuit, fpga_config, qchip)

In [9]:
compiled_prog

CompiledProgram(program={('Q0.qdrv', 'Q0.rdrv', 'Q0.rdlo'): [{'op': 'phase_reset'}, {'op': 'declare_freq', 'freq': np.float64(4493000000.0), 'channel': 'Q0.qdrv', 'freq_ind': 0}, {'op': 'declare_freq', 'freq': np.float64(4493068965.5172415), 'channel': 'Q0.qdrv', 'freq_ind': 1}, {'op': 'declare_freq', 'freq': np.float64(4493137931.034483), 'channel': 'Q0.qdrv', 'freq_ind': 2}, {'op': 'declare_freq', 'freq': np.float64(4493206896.551724), 'channel': 'Q0.qdrv', 'freq_ind': 3}, {'op': 'declare_freq', 'freq': np.float64(4493275862.068966), 'channel': 'Q0.qdrv', 'freq_ind': 4}, {'op': 'declare_freq', 'freq': np.float64(4493344827.586206), 'channel': 'Q0.qdrv', 'freq_ind': 5}, {'op': 'declare_freq', 'freq': np.float64(4493413793.103448), 'channel': 'Q0.qdrv', 'freq_ind': 6}, {'op': 'declare_freq', 'freq': np.float64(4493482758.620689), 'channel': 'Q0.qdrv', 'freq_ind': 7}, {'op': 'declare_freq', 'freq': np.float64(4493551724.137931), 'channel': 'Q0.qdrv', 'freq_ind': 8}, {'op': 'declare_freq